In [ ]:
import sys
sys.path.append("../")

import tables
import numpy as np
import h5py
from pathlib import Path
import time

from kdmc.utils import compute_sample_energy

In [ ]:
def compute_snr(signal, noise):
    """Computes the SNR of one/multiple signal (assumes last two dimensions are the 
    iq channels and time samples)"""
    if signal.shape != noise.shape:
        raise ValueError("Signal and noise must have the same shape")
    return 10 * np.log10(np.mean(signal ** 2, axis=(-1,-2)) / np.mean(noise ** 2, axis=(-1,-2)))

In [ ]:
data_path = Path("../../data/signal")
filename = "srml2018_4.mat"

st = time.process_time()

with h5py.File(data_path.joinpath(filename), 'r', rdcc_nbytes=1024**2*4000, rdcc_w0=1) as f:
    snr = np.squeeze(f['snrs'][:])
    rx_x = f['rx_x'][:]
    rx_x = np.swapaxes(rx_x, 0, 2)
    rx_x = rx_x.astype(np.float32)
    # Normalize rx_x
    rx_x = rx_x / compute_sample_energy(rx_x)[:, np.newaxis, np.newaxis]
    rx_s = f['rx_s'][:]
    rx_s = np.swapaxes(rx_s, 0, 2)
    y = f['y'][:]
    y = np.swapaxes(y, 0, 1)
    y = y.astype(np.float32)

    modulation = np.argmax(y, axis=1)
    modulation = modulation.astype(np.int64)

    sps = np.squeeze(f['lsps'][:])
    rolloff = np.squeeze(f['rolloffs'][:])
    fs = np.squeeze(f['fss'][:])
    channel = np.squeeze(f['rays'][:])
    snrs_filt = np.squeeze(f['snrs_filt'][:])
    
et = time.process_time()
print('CPU Execution time:', et - st, 'seconds')

In [ ]:
with open(data_path.joinpath('rx_x.npy'), 'wb') as f:
    np.save(f, rx_x)
with open(data_path.joinpath('rx_s.npy'), 'wb') as f:
    np.save(f, rx_s)
with open(data_path.joinpath('y.npy'), 'wb') as f:
    np.save(f, y)
with open(data_path.joinpath('snr.npy'), 'wb') as f:
    np.save(f, snr)
with open(data_path.joinpath('snr_filt.npy'), 'wb') as f:
    np.save(f, snrs_filt)
with open(data_path.joinpath('sps.npy'), 'wb') as f:
    np.save(f, sps)
with open(data_path.joinpath('rolloff.npy'), 'wb') as f:
    np.save(f, rolloff)
with open(data_path.joinpath('fs.npy'), 'wb') as f:
    np.save(f, fs)
with open(data_path.joinpath('channel.npy'), 'wb') as f:
    np.save(f, channel)

In [ ]:
st = time.process_time()

with open(data_path.joinpath('rx_x.npy'), 'rb') as f:
    rx_x = np.load(f)
with open(data_path.joinpath('rx_s.npy'), 'rb') as f:
    rx_s = np.load(f)
with open(data_path.joinpath('y.npy'), 'rb') as f:
    y = np.load(f)
with open(data_path.joinpath('snr.npy'), 'rb') as f:
    snr = np.load(f)
with open(data_path.joinpath('snr_filt.npy'), 'rb') as f:
    snr_filt = np.load(f)
with open(data_path.joinpath('sps.npy'), 'rb') as f:
    sps = np.load(f)
with open(data_path.joinpath('rolloff.npy'), 'rb') as f:
    rolloff = np.load(f)
with open(data_path.joinpath('fs.npy'), 'rb') as f:
    fs = np.load(f)
with open(data_path.joinpath('channel.npy'), 'rb') as f:
    channel = np.load(f)
    
et = time.process_time()
print('CPU Execution time:', et - st, 'seconds')